# Notebook 8: Agent con Query Database

**Obiettivo**: Implementare un Agent che può eseguire query su database (SQL) per recuperare informazioni real-time

**Sicurezza**: Questo notebook enfatizza la prevenzione di SQL injection. In produzione, usa sempre parametri preparati!

**Nota**: Questo notebook usa SQLite per semplicità. In produzione, si può usare PostgreSQL, MySQL, etc.

---


## 1. Setup Database Esempio

Creiamo un database SQLite di esempio con dati simulati per testare l'integrazione.


In [ ]:
import sqlite3
import os

# Crea database SQLite (o usa esistente)
db_path = "example_helpdesk.db"

# Rimuovi database esistente se presente (per test)
if os.path.exists(db_path):
    os.remove(db_path)
    print("🗑️ Database esistente rimosso")

# Connetti al database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Crea tabella utenti
cursor.execute("""
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT NOT NULL,
    email TEXT NOT NULL,
    role TEXT NOT NULL,
    created_at TEXT NOT NULL
)
""")

# Crea tabella ticket
cursor.execute("""
CREATE TABLE IF NOT EXISTS tickets (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    ticket_id TEXT UNIQUE NOT NULL,
    title TEXT NOT NULL,
    description TEXT,
    status TEXT NOT NULL,
    priority TEXT NOT NULL,
    user_id INTEGER,
    created_at TEXT NOT NULL,
    FOREIGN KEY (user_id) REFERENCES users(id)
)
""")

# Inserisci dati di esempio
from datetime import datetime

# Utenti
users_data = [
    ("mario.rossi", "mario.rossi@example.com", "user", datetime.now().isoformat()),
    ("luigi.verdi", "luigi.verdi@example.com", "admin", datetime.now().isoformat()),
    ("anna.bianchi", "anna.bianchi@example.com", "user", datetime.now().isoformat()),
]

cursor.executemany(
    "INSERT INTO users (username, email, role, created_at) VALUES (?, ?, ?, ?)",
    users_data
)

# Ticket
tickets_data = [
    ("TICKET-0001", "Problema login", "Non riesco ad accedere", "open", "high", 1, datetime.now().isoformat()),
    ("TICKET-0002", "Richiesta informazioni", "Vorrei sapere gli orari", "resolved", "medium", 2, datetime.now().isoformat()),
    ("TICKET-0003", "Bug sistema", "Il sistema si blocca", "in_progress", "urgent", 1, datetime.now().isoformat()),
]

cursor.executemany(
    "INSERT INTO tickets (ticket_id, title, description, status, priority, user_id, created_at) VALUES (?, ?, ?, ?, ?, ?, ?)",
    tickets_data
)

# Salva modifiche
conn.commit()

print("✅ Database creato e popolato!")
print(f"  Database: {db_path}")
print(f"  Tabelle: users, tickets")
print(f"  Utenti: {len(users_data)}")
print(f"  Ticket: {len(tickets_data)}")


## 2. Test Database

Verifichiamo che il database funzioni correttamente con query semplici.


In [ ]:
# Test query utenti
cursor.execute("SELECT * FROM users")
users = cursor.fetchall()
print("Utenti nel database:")
for user in users:
    print(f"  - {user[1]} ({user[2]}) - {user[3]}")

# Test query ticket
cursor.execute("SELECT * FROM tickets")
tickets = cursor.fetchall()
print("\nTicket nel database:")
for ticket in tickets:
    print(f"  - {ticket[1]}: {ticket[2]} ({ticket[4]}, {ticket[5]})")

print("\n✅ Database funziona correttamente!")


## 3. Setup LangChain e LLM

Importiamo LangChain e configuriamo l'LLM per l'Agent.


In [ ]:
# LangChain 1.2+
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

# Inizializza LLM
llm = ChatOllama(
    model="llama3.2:3b",
    temperature=0
)

print("✅ LangChain e LLM configurati!")


## 4. Creare Tools Database

**ATTENZIONE!**: Usiamo sempre parametri preparati per prevenire SQL injection!


In [ ]:
# Tool 1: Query database generica (SICURA - solo SELECT)
@tool
def query_database(query: str) -> str:
    """
    Esegue una query SQL raw (SOLO SELECT) sul database SQLite per analisi complesse e personalizzate.
    Utilizza questo tool quando le altre funzioni specifiche non sono sufficienti.

    SCHEMA DEL DATABASE:
    1. Tabella 'users':
       - id (INTEGER, Primary Key)
       - username (TEXT)
       - email (TEXT)
       - role (TEXT: 'admin', 'user', 'support')
       - created_at (DATETIME)

    2. Tabella 'tickets':
       - ticket_id (INTEGER, Primary Key)
       - user_id (INTEGER, Foreign Key -> users.id)
       - title (TEXT)
       - description (TEXT)
       - status (TEXT: 'open', 'in_progress', 'resolved', 'closed')
       - priority (TEXT: 'low', 'medium', 'high', 'critical')
       - created_at (DATETIME)

    REGOLE IMPORTANTI:
    - SINTASSI: Usa sintassi SQLite standard.
    - SICUREZZA: Sono permesse SOLO operazioni di lettura (SELECT). Qualsiasi tentativo di modifica (INSERT, UPDATE, DELETE, DROP) verrà bloccato.
    - JOIN: Puoi eseguire JOIN tra 'users' e 'tickets' usando 'users.id' e 'tickets.user_id'.

    Args:
        query: La stringa SQL completa da eseguire (es. "SELECT * FROM tickets LIMIT 5")

    Returns:
        Stringa formattata con i risultati della query o messaggio di errore.
    """
    # SICUREZZA: Verifica che sia una query SELECT
    query_upper = query.strip().upper()
    if not query_upper.startswith("SELECT"):
        return "Errore: Solo query SELECT sono permesse. Non è possibile modificare dati."

    # SICUREZZA: Blocca comandi pericolosi
    dangerous_keywords = ["DROP", "DELETE", "UPDATE", "INSERT", "ALTER", "CREATE", "TRUNCATE"]
    for keyword in dangerous_keywords:
        if keyword in query_upper:
            return f"Errore: Comando '{keyword}' non permesso. Solo SELECT è consentito."

    try:
        # Esegui query (SICURO: query è validata sopra)
        cursor.execute(query)
        results = cursor.fetchall()
        conn.commit()

        if not results:
            return "Nessun risultato trovato."

        # Formatta risultati
        result_str = f"Trovati {len(results)} risultati:\n"
        for row in results:
            result_str += f"  {row}\n"

        return result_str
    except Exception as e:
        return f"Errore nella query: {str(e)}"

# Tool 2: Query utenti per username (SICURA - parametri preparati)
@tool
def get_user_by_username(username: str) -> str:
    """
    Cerca e recupera i dettagli completi di un singolo utente specificando il suo username esatto.

    Utilizza questo tool quando devi identificare una persona specifica o verificare i suoi permessi/ruoli.
    Non supporta ricerche parziali (wildcards), l'username deve corrispondere esattamente.

    OUTPUT RESTITUITO:
    - Username
    - Email di contatto
    - Ruolo nel sistema (es. admin, user)
    - Data di creazione account

    Args:
        username: L'username esatto da cercare (es. "mario.rossi")

    Returns:
        Stringa contenente i dettagli dell'utente o messaggio di non trovato.
    """
    try:
        cursor.execute("SELECT * FROM users WHERE username = ?", (username,))
        user = cursor.fetchone()

        if user:
            return f"Utente: {user[1]} | Email: {user[2]} | Ruolo: {user[3]} | Creato: {user[4]}"
        else:
            return f"Utente '{username}' non trovato"
    except Exception as e:
        return f"Errore: {str(e)}"

# Tool 3: Query ticket per stato (SICURA - parametri preparati)
@tool
def get_tickets_by_status(status: str) -> str:
    """
    Filtra e recupera la lista dei ticket che si trovano in uno specifico stato di lavorazione.

    Utilizza questo tool per monitorare il flusso di lavoro, ad esempio per vedere tutti i ticket ancora aperti o quelli già chiusi.

    VALORI AMMESSI PER 'STATUS':
    - 'open': Ticket aperti in attesa di presa in carico.
    - 'in_progress': Ticket attualmente in lavorazione.
    - 'resolved': Ticket risolti ma in attesa di conferma.
    - 'closed': Ticket archiviati definitivamente.

    Args:
        status: Lo stato del ticket da filtrare (scegliere ESCLUSIVAMENTE tra i valori ammessi).

    Returns:
        Lista di ticket (ID, Titolo, Priorità) corrispondenti allo stato richiesto.
    """
    try:
        cursor.execute(
            "SELECT ticket_id, title, status, priority FROM tickets WHERE status = ?",
            (status,)
        )
        tickets = cursor.fetchall()

        if not tickets:
            return f"Nessun ticket trovato con stato '{status}'"

        result = f"Trovati {len(tickets)} ticket con stato '{status}':\n"
        for ticket in tickets:
            result += f"  - ID {ticket[0]}: {ticket[1]} (Priorità: {ticket[3]})\n"

        return result
    except Exception as e:
        return f"Errore: {str(e)}"

# Tool 4: Statistiche database (SICURA)
@tool
def get_database_stats() -> str:
    """
    Fornisce una panoramica ad alto livello (Dashboard) dello stato del database.

    Utilizza questo tool come PRIMO PASSO per capire la dimensione dei dati e la distribuzione del carico di lavoro
    prima di effettuare query più specifiche.

    INFORMAZIONI RESTITUITE:
    1. Conteggio totale degli utenti registrati.
    2. Distribuzione dei ticket per stato (quanti open, quanti closed, etc.).

    Returns:
        Report statistico testuale riassuntivo.
    """
    try:
        # Conta utenti
        cursor.execute("SELECT COUNT(*) FROM users")
        user_count = cursor.fetchone()[0]

        # Conta ticket per stato
        cursor.execute("SELECT status, COUNT(*) FROM tickets GROUP BY status")
        ticket_stats = cursor.fetchall()

        result = f"Statistiche Database (Dashboard):\n"
        result += f"  Utenti totali registrati: {user_count}\n"
        result += f"  Riepilogo Ticket per Stato:\n"
        for stat in ticket_stats:
            result += f"    - {stat[0]}: {stat[1]}\n"

        return result
    except Exception as e:
        return f"Errore: {str(e)}"

In [ ]:
# Test query generica
result1 = query_database.invoke("SELECT * FROM users")
print(f"Test 1 - Query generica:\n{result1}\n")

# Test query utente
result2 = get_user_by_username.invoke({"username": "mario.rossi"})
print(f"Test 2 - Query utente:\n{result2}\n")

# Test query ticket
result3 = get_tickets_by_status.invoke({"status": "open"})
print(f"Test 3 - Query ticket:\n{result3}\n")

# Test statistiche
result4 = get_database_stats.invoke({})
print(f"Test 4 - Statistiche:\n{result4}\n")

print("✅ Tutti i tools Database funzionano!")


## 6. Dimostrazione SQL Injection (NON SICURA)

**ATTENZIONE**: Questo esempio mostra cosa NON fare! Usa sempre parametri preparati.


In [ ]:
# Query con string formatting (vulnerabile a SQL injection)
def unsafe_query(username):
    """NON USARE QUESTO APPROCCIO!"""
    query = f"SELECT * FROM users WHERE username = '{username}'"
    # Se username = "admin' OR '1'='1", la query diventa:
    # SELECT * FROM users WHERE username = 'admin' OR '1'='1'
    # Questo restituisce TUTTI gli utenti!
    return query

# Esempio di attacco SQL injection
malicious_input = "admin' OR '1'='1"
unsafe_query_example = unsafe_query(malicious_input)
print(f"⚠️ Query NON SICURA: {unsafe_query_example}")
print("Questa query restituirebbe TUTTI gli utenti, non solo 'admin'!\n")

# Query con parametri preparati (SICURO)
def safe_query(username):
    # Usa parametri preparati (? placeholder)
    query = "SELECT * FROM users WHERE username = ?"
    # Il database gestisce l'escape automaticamente
    return query, (username,)

safe_query_example, params = safe_query(malicious_input)
print(f"✅ Query SICURA: {safe_query_example}")
print(f"   Parametri: {params}")
print("Questa query è SICURA anche con input maliziosi!")

print("\n💡 REGOLA D'ORO: Usa SEMPRE parametri preparati per input utente!")


## 7. Creare Agent Database

Creiamo un Agent ReAct che può usare i tools database per recuperare informazioni.


In [ ]:
# Raccogli tutti i tools
tools = [query_database, get_user_by_username, get_tickets_by_status, get_database_stats]

# Crea system prompt per Agent Database (Dettagliato e Ottimizzato)
system_prompt = """Sei un **Senior Database Administrator (DBA)** AI specializzato in analisi dati e supporto tecnico.
Il tuo compito è interrogare il database aziendale per fornire risposte precise, sicure e ben formattate agli utenti.

### 1. CONOSCENZA DEL DATABASE (Schema)
Hai accesso a un database SQLite con le seguenti tabelle:
* **users**: `id` (int), `username` (text), `email` (text), `role` (text), `created_at` (datetime).
* **tickets**: `ticket_id` (int), `user_id` (int, FK), `title` (text), `description` (text), `status` (text: 'open', 'in_progress', 'resolved', 'closed'), `priority` (text), `created_at` (datetime).

### 2. STRATEGIA DI UTILIZZO DEI TOOLS
Devi seguire questa gerarchia decisionale rigorosa prima di agire:

1.  **Analisi Generale:** Se l'utente chiede "come va il sistema?" o statistiche globali, usa PRIMA `get_database_stats`.
2.  **Ricerche Specifiche:** Se l'utente cerca un'entità specifica (es. "chi è l'utente X?" o "fammi vedere i ticket aperti"), usa SEMPRE i tool specifici (`get_user_by_username`, `get_tickets_by_status`). Sono più veloci e sicuri.
3.  **Query Complesse:** Usa `query_database` (SQL generico) SOLO se la richiesta non è soddisfacibile dai tool specifici (es. JOIN tra utenti e ticket, raggruppamenti complessi, filtri su date).
    * *Esempio valido per SQL:* "Quanti ticket 'critical' ha aperto l'utente Mario?"
    * *Esempio non valido:* "Cerca l'utente Mario" (Usa `get_user_by_username`).

### 3. PROTOCOLLI DI SICUREZZA
* **READ-ONLY:** Non tentare mai di modificare, cancellare o inserire dati. I tuoi tool bloccheranno queste operazioni, ma tu non devi nemmeno provarci.
* **Privacy:** Se un tool restituisce dati sensibili (password hash, token), non mostrarli mai nell'output finale.

### 4. FORMATTAZIONE OUTPUT
* Non restituire mai raw data (liste/tuple Python) direttamente all'utente.
* Usa elenchi puntati o tabelle Markdown per presentare i dati.
* Se non trovi risultati, rispondi chiaramente: "Nessun dato trovato per i criteri specificati".
* Sii conciso ma professionale.

### OBIETTIVO
Rispondi alla domanda dell'utente nel modo più accurato possibile usando i dati recuperati.
"""

# Crea Agent
# Nota: Assicurati che la funzione create_agent supporti il parametro system_prompt.
# Se usi create_react_agent standard, dovrai inserire questo system_prompt nel template del prompt.
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=system_prompt,
    debug=True
)


print("✅ Agent Database creato con System Prompt avanzato!")
print(f"Tools disponibili: {len(tools)}")
print("⚠️ Tutti i tools sono SICURI (parametri preparati o validazione)")

## 8. Test Agent Database

Testiamo l'Agent con query reali sul database.


In [ ]:
# Test 1: Query utente
query1 = "Dimmi le informazioni sull'utente mario.rossi"
print(f"Query: {query1}\n")
result1 = agent.invoke({"messages": [HumanMessage(content=query1)]})
print(f"\nRisposta: {result1['messages'][-1].content}\n")
print("="*60)


In [ ]:
# Test 2: Query ticket
query2 = "Quanti ticket aperti ci sono sul database?"
print(f"Query: {query2}\n")
result2 = agent.invoke({"messages": [HumanMessage(content=query2)]})
print(f"\nRisposta: {result2['messages'][-1].content}\n")
print("="*60)


In [ ]:
# Test 3: Statistiche
query3 = "Mostrami le statistiche del database"
print(f"Query: {query3}\n")
result3 = agent.invoke({"messages": [HumanMessage(content=query3)]})
print(f"\nRisposta: {result3['messages'][-1].content}\n")
print("="*60)


In [ ]:
# Test 4: Query complessa
query4 = "Mostrami tutti i ticket in progress con priorità urgente"
print(f"Query: {query4}\n")
result4 = agent.invoke({"messages": [HumanMessage(content=query4)]})
print(f"\nRisposta: {result4['messages'][-1].content}\n")
print("="*60)


## 9. Note e Best Practices

### Cosa abbiamo imparato:
1. **Database Integration**: Come integrare Agent con database SQL
2. **SQL Injection Prevention**: Usare parametri preparati SEMPRE
3. **Query Validation**: Validare query per permettere solo SELECT
4. **Tool Safety**: Tools devono essere sicuri per default
5. **Real-time Data**: Agent può accedere a dati real-time dal database

### Best Practices Sicurezza:

**⚠️ CRITICO - SQL Injection Prevention**:
- **SEMPRE** usa parametri preparati per input utente
- **NON** usare string formatting (f-strings, .format()) per query
- **Valida** query per permettere solo SELECT
- **Blocca** comandi pericolosi (DROP, DELETE, UPDATE, etc.)

**Database Permissions**:
- Limita permessi database (solo SELECT se possibile)
- Usa utente database con permessi minimi necessari
- Non usare account admin per applicazioni

**Error Handling**:
- Gestisci errori database gracefully
- Non esporre dettagli errori SQL agli utenti
- Log errori per debugging

### Limitazioni e Considerazioni:
- **Performance**: Query complesse possono essere lente
- **Connection Pooling**: In produzione, usa connection pooling
- **Read-Only**: Questo notebook mostra solo query (SELECT)
- **Production**: In produzione, usa database reali (PostgreSQL, MySQL)

---

**Congratulazioni! Hai completato il Notebook 8! 🎉**
